df_fin -> input dataframe com os dados financeiros das empresas

df_cod -> input dataframe com os códigos CVM dos ativos

df_mag -> output dataframe que irá armazenar o resultado com as magic stocks

In [1]:
from pathlib import Path
import pandas as pd

# Mostrar floats com duas casas decimas
pd.set_option('display.float_format',  lambda x: '%.2f' % x)
pd.options.display.max_colwidth = 20
pd.options.display.max_columns = 20
pd.options.display.max_rows = 6

In [2]:
# Carregar base NÃO AJUSTADA do S3 na seguinte raíz: s3://aq-dl/HistoricalQuotations/
# Caminho local:
DATA_FOLDER = Path("/mnt/aq_disk/data/HistoricalQuotations/processed")
# BASE_ADJ = DATA_FOLDER / "base_adj.feather"
BASE1 = DATA_FOLDER / "base_95-21.feather"
BASE2 = DATA_FOLDER / "base_22.feather"
cols = ['datneg', 'codneg', 'nomres', 'especi', 'codbdi', 'tpmerc', 'preult', 'premed', 'totneg', 'voltot']
df_mag = (pd
    .concat([pd.read_feather(BASE1), pd.read_feather(BASE2)], ignore_index=True)
    [cols]
    .query('tpmerc == 10')
)
df_mag

,datneg,codneg,nomres,especi,codbdi,tpmerc,preult,premed,totneg,voltot
0,2020-02-07,A1AP34,ADVANCE AUTO,DRN,2,10,143.12,143.12,2,6139848.00
1,2020-02-10,A1AP34,ADVANCE AUTO,DRN,2,10,142.27,142.27,1,512172.00
2,2020-02-13,A1AP34,ADVANCE AUTO,DRN,2,10,147.37,147.37,1,235792.00
...,...,...,...,...,...,...,...,...,...,...
10199338,2022-06-17,GOLL11,GOL,BNS PRE N2,22,10,2.15,2.04,12,10860.00
10199339,2022-06-17,TASA17,TAURUS ARMAS,BNS PRE N2,22,10,10.40,10.58,5,6351.00
10199340,2022-06-17,VLID11,VALID,BNS ORD NM,22,10,0.64,0.63,4,2105.00


#### Filtrar:
1. Cotações após 2011
2. Lote padrão (codbdi == 2) -> remove negociações de empresas em concordata, recuperação judicial, etc.
3. Ações ON ou PN

In [3]:
df_mag.query('\
    codbdi == 2 and \
    datneg >= "2011.01.01" and \
    especi.str.contains("ON |PN ")'
    , inplace=True
)
df_mag.reset_index(drop=True, inplace=True)
df_mag

,datneg,codneg,nomres,especi,codbdi,tpmerc,preult,premed,totneg,voltot
0,2016-10-28,AALR3,ALLIAR,ON NM,2,10,19.20,19.28,4460,122334647.00
1,2016-10-31,AALR3,ALLIAR,ON NM,2,10,18.06,18.17,4238,45857231.00
2,2016-11-01,AALR3,ALLIAR,ON NM,2,10,17.90,17.74,2072,17676981.00
...,...,...,...,...,...,...,...,...,...,...
578461,2022-06-17,WEST3,WESTWING,ON NM,2,10,2.11,2.10,4082,1215847.00
578462,2022-06-17,WIZS3,WIZ S.A.,ON NM,2,10,7.82,7.76,2964,6927985.00
578463,2022-06-17,YDUQ3,YDUQS PART,ON NM,2,10,13.33,13.31,16766,73538234.00


In [4]:
# Manter somente as colunas que serão usadas para fazer o corte nas datas
cols = ['datneg', 'codneg', 'nomres', 'premed', 'totneg']
df_mag = df_mag.loc[:, cols]
# Inserir o cód. dos emissores -> 4 primeiros caracteres do código de negociação
df_mag['codemi'] = df_mag.codneg.str[0:4]
# Inserir o dia do ano e o ano para a operação de corte das ações
df_mag['day_year'] = df_mag.datneg.dt.day_of_year
df_mag['year'] = df_mag.datneg.dt.year
print('Number of companies available for backtesting', df_mag.codemi.nunique())
df_mag

Number of companies available for backtesting 500


,datneg,codneg,nomres,premed,totneg,codemi,day_year,year
0,2016-10-28,AALR3,ALLIAR,19.28,4460,AALR,302,2016
1,2016-10-31,AALR3,ALLIAR,18.17,4238,AALR,305,2016
2,2016-11-01,AALR3,ALLIAR,17.74,2072,AALR,306,2016
...,...,...,...,...,...,...,...,...
578461,2022-06-17,WEST3,WESTWING,2.10,4082,WEST,168,2022
578462,2022-06-17,WIZS3,WIZ S.A.,7.76,2964,WIZS,168,2022
578463,2022-06-17,YDUQ3,YDUQS PART,13.31,16766,YDUQ,168,2022


In [5]:
# Definir o dia de corte e remover negociações posteriores ao corte
df_mag.query('day_year >= 100', inplace=True)
df_mag.reset_index(drop=True, inplace=True)
df_mag

,datneg,codneg,nomres,premed,totneg,codemi,day_year,year
0,2016-10-28,AALR3,ALLIAR,19.28,4460,AALR,302,2016
1,2016-10-31,AALR3,ALLIAR,18.17,4238,AALR,305,2016
2,2016-11-01,AALR3,ALLIAR,17.74,2072,AALR,306,2016
...,...,...,...,...,...,...,...,...
414633,2022-06-17,WEST3,WESTWING,2.10,4082,WEST,168,2022
414634,2022-06-17,WIZS3,WIZ S.A.,7.76,2964,WIZS,168,2022
414635,2022-06-17,YDUQ3,YDUQS PART,13.31,16766,YDUQ,168,2022


In [6]:
# Calcular o dia mais próximo de 100 em cada ano
df_corte = df_mag.groupby(by=['year'])['day_year'].min().reset_index()
df_corte

,year,day_year
0,2011,101
1,2012,100
2,2013,100
...,...,...
9,2020,100
10,2021,102
11,2022,101


In [7]:
# Filtrar o dataframe de ações com dados somente nos dias de corte
# Usar 'year' e 'day_year' como chaves para a operação de união entre os dois dataframes
df_mag = df_mag.merge(
    right=df_corte, how='inner', on=['year', 'day_year']
)
# A coluna 'day_year' não será mais usada
df_mag.drop(columns=['day_year'], inplace=True)
print('Number of companies available for backtesting:', df_mag.codemi.nunique())
df_mag

Number of companies available for backtesting: 377


,datneg,codneg,nomres,premed,totneg,codemi,year
0,2017-04-10,AALR3,ALLIAR,15.36,315,AALR,2017
1,2017-04-10,ABCB4,ABC BRASIL,18.36,1005,ABCB,2017
2,2017-04-10,AGRO3,BRASILAGRO,12.29,199,AGRO,2017
...,...,...,...,...,...,...,...
2464,2022-04-11,WIZS3,WIZ S.A.,8.49,6873,WIZS,2022
2465,2022-04-11,WLMM4,WLM IND COM,34.55,14,WLMM,2022
2466,2022-04-11,YDUQ3,YDUQS PART,19.71,10252,YDUQ,2022


In [8]:
# Excluir empresas financeiras e prestadoras de serviços públicos (utilities) conforme
# lista extraída do seguinte arquivo da B3, sem versionamento:
# https://bvmf.bmfbovespa.com.br/InstDados/InformacoesEmpresas/ClassifSetorial.zip
excluded_companies = pd.read_csv('../data/external/excluded_companies.csv')
excluded_companies = excluded_companies['company_code'].to_list()
df_mag.query('codemi != @excluded_companies', inplace=True)
print('Number of companies available for backtesting', df_mag.codemi.nunique())
df_mag

Number of companies available for backtesting 311


,datneg,codneg,nomres,premed,totneg,codemi,year
0,2017-04-10,AALR3,ALLIAR,15.36,315,AALR,2017
2,2017-04-10,AGRO3,BRASILAGRO,12.29,199,AGRO,2017
3,2017-04-10,ALPA3,ALPARGATAS,10.59,3,ALPA,2017
...,...,...,...,...,...,...,...
2463,2022-04-11,WEST3,WESTWING,2.92,2632,WEST,2022
2465,2022-04-11,WLMM4,WLM IND COM,34.55,14,WLMM,2022
2466,2022-04-11,YDUQ3,YDUQS PART,19.71,10252,YDUQ,2022


In [9]:
# Carregar dataframe com o código de emissão das empresas
df_cod = pd.read_pickle('/mnt/aq_disk/data/AQ/codemi.pkl')
df_cod

,codcvm,cnpj,densoc,situac,codemi
0,60,18451005000104,ACOPALMA CIA IND...,CANCELADA,ZWVZ
1,94,92693019000189,PANATLANTICA SA,ATIVO,PATI
2,108,60664810000174,AÇOS VILLARES SA,CANCELADA,AVIL
...,...,...,...,...,...
1763,26808,04368898000106,COPEL DISTRIBUIÇ...,ATIVO,CPLD
1764,26824,43335774000186,TRAVESSIA SECURI...,ATIVO,TMER
1765,26832,38482780000126,ANEMUS WIND HOLD...,ATIVO,ANEM


In [10]:
# A chave da união será o cód. do emissor (codemi)
# Somente o cód. CVM (codcvm) será inserido na união -> Remover colunas que não serão 
# usadas na operação de merge
df_cod = df_cod[['codcvm', 'codemi']].copy()
df_cod

,codcvm,codemi
0,60,ZWVZ
1,94,PATI
2,108,AVIL
...,...,...
1763,26808,CPLD
1764,26824,TMER
1765,26832,ANEM


In [11]:
# Criar um set com a lista de ativos antes da operação de união dos dataframes
s0 = set(df_mag.codemi.unique())
# Obter os códigos de emissão dos ativos unindo os dataframes 
df_mag = df_mag.merge(right=df_cod, how='inner', on='codemi')
df_mag.reset_index(drop=True, inplace=True)
print('Number of companies available for backtesting:', df_mag.codemi.nunique())
df_mag

Number of companies available for backtesting: 277


,datneg,codneg,nomres,premed,totneg,codemi,year,codcvm
0,2017-04-10,AALR3,ALLIAR,15.36,315,AALR,2017,24058
1,2018-04-10,AALR3,ALLIAR,15.17,175,AALR,2018,24058
2,2019-04-10,AALR3,ALLIAR,14.64,156,AALR,2019,24058
...,...,...,...,...,...,...,...,...
1796,2022-04-11,VIIA3,VIA,3.57,30458,VIIA,2022,6505
1797,2022-04-11,VITT3,VITTIA,12.26,232,VITT,2022,25763
1798,2022-04-11,VVEO3,VIVEO,15.75,871,VVEO,2022,25682


In [12]:
# Verificar a diferença entre os dois sets
s1 = set(df_mag.codemi.unique())
print('Núm. de empresas cujo código não foi localizado', len(s0-s1))
# 34 empresas não tiveram seu código localizado. Analisando os dados, tratam-se de
# empresas cujo código de listagem foi alterado: BVMF->B3SA, VVAR->VIIA, etc
print(s0 - s1)

Núm. de empresas cujo código não foi localizado 34
{'MPXE', 'ENMA', 'RNAR', 'SSBR', 'SNSL', 'FJTA', 'BVMF', 'BRDT', 'TBLE', 'ESTC', 'HRTP', 'CTAX', 'ECOD', 'ABRE', 'OHLB', 'BPNM', 'LIQO', 'BRIN', 'DROG', 'BBRK', 'DTEX', 'KROT', 'PARC', 'CNTO', 'QGEP', 'ALLL', 'CCPR', 'IDNT', 'PRTX', 'VAGR', 'LLXL', 'BTOW', 'INPR', 'VVAR'}


In [13]:
# Carregar dataframe com os dados financeiros das empresas
df_fin = pd.read_csv(
    '../data/1_financials.csv',
    parse_dates=['doc_env', 'per_ini', 'per_fim']
)
# Renomear coluna com o código CVM para coincidir com as outras bases
df_fin.rename(columns={'cia_id': 'codcvm'}, inplace=True)
# As colunas 'per_ini' e 'cia_nome' não serão usadas
df_fin.drop(columns=['per_ini', 'cia_nome'], inplace=True)
# year = ano em que a informação será usada -> ano seguinte ao fim do período
df_fin['year'] = df_fin.per_fim.dt.year + 1
df_fin

,codcvm,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,year
0,94,2011-03-31 10:16:48,2010-12-31,8856000.00,-18981000.00,24777000.00,0.20,2011
1,94,2011-04-01 17:31:56,2010-12-31,8856000.00,-18980000.00,21093000.00,0.17,2011
2,94,2012-03-20 15:20:37,2011-12-31,9480000.00,-46182000.00,8720000.00,0.08,2012
...,...,...,...,...,...,...,...,...
4055,80187,2022-02-24 23:23:28,2021-12-31,72244618.00,-24569000.00,627642000.00,0.42,2022
4056,80195,2021-03-12 18:33:08,2020-12-31,66025325.00,103505000.00,92232000.00,0.19,2021
4057,90212,2018-05-30 15:43:03,2017-12-31,216075329.00,-70761000.00,241375000.00,0.31,2018


In [14]:
# Incluir os dados contábeis em 'df_mag' 
df_mag = df_mag.merge(right=df_fin, how='inner', on=['year', 'codcvm'])
# A coluna 'codcvm não será mais usada
df_mag.drop(columns=['codcvm'], inplace=True)
df_mag

,datneg,codneg,nomres,premed,totneg,codemi,year,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic
0,2017-04-10,AALR3,ALLIAR,15.36,315,AALR,2017,2017-03-22 23:42:13,2016-12-31,114898767.00,308503000.00,100612000.00,0.07
1,2018-04-10,AALR3,ALLIAR,15.17,175,AALR,2018,2018-03-28 20:20:31,2017-12-31,118292816.00,507645000.00,70337000.00,0.04
2,2019-04-10,AALR3,ALLIAR,14.64,156,AALR,2019,2019-03-19 19:58:58,2018-12-31,118292816.00,532346000.00,137286000.00,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,2022-04-11,VITT3,VITTIA,12.26,232,VITT,2022,2022-03-18 12:29:36,2021-12-31,143049107.00,159804000.00,142840000.00,0.22
1949,2022-04-11,VVEO3,VIVEO,15.75,871,VVEO,2022,2022-03-10 18:22:56,2021-12-31,286123360.00,-76527000.00,492300000.00,0.24
1950,2022-04-11,VVEO3,VIVEO,15.75,871,VVEO,2022,2022-03-30 18:25:34,2021-12-31,286123360.00,-76527000.00,492300000.00,0.24


In [15]:
# Converter os dados contábeis para milhões para facilitar verificações posteriores
accounting_columns = ['shares_outstanding', 'net_debt', 'ebit']
df_mag[accounting_columns] = df_mag[accounting_columns] / 1_000_000
df_mag

,datneg,codneg,nomres,premed,totneg,codemi,year,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic
0,2017-04-10,AALR3,ALLIAR,15.36,315,AALR,2017,2017-03-22 23:42:13,2016-12-31,114.90,308.50,100.61,0.07
1,2018-04-10,AALR3,ALLIAR,15.17,175,AALR,2018,2018-03-28 20:20:31,2017-12-31,118.29,507.64,70.34,0.04
2,2019-04-10,AALR3,ALLIAR,14.64,156,AALR,2019,2019-03-19 19:58:58,2018-12-31,118.29,532.35,137.29,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,2022-04-11,VITT3,VITTIA,12.26,232,VITT,2022,2022-03-18 12:29:36,2021-12-31,143.05,159.80,142.84,0.22
1949,2022-04-11,VVEO3,VIVEO,15.75,871,VVEO,2022,2022-03-10 18:22:56,2021-12-31,286.12,-76.53,492.30,0.24
1950,2022-04-11,VVEO3,VIVEO,15.75,871,VVEO,2022,2022-03-30 18:25:34,2021-12-31,286.12,-76.53,492.30,0.24


In [16]:
# Calcular os indicadores que dependem do preço do ação
df_mag['market_cap'] = df_mag['shares_outstanding'] * df_mag['premed']
df_mag['enterprise_value'] = df_mag['market_cap'] - df_mag['net_debt']
df_mag['earnings_yield'] = df_mag['ebit'] / df_mag['enterprise_value']
# A coluna 'premed' não será mais usada
df_mag.drop(columns=['premed'], inplace=True)
df_mag

,datneg,codneg,nomres,totneg,codemi,year,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield
0,2017-04-10,AALR3,ALLIAR,315,AALR,2017,2017-03-22 23:42:13,2016-12-31,114.90,308.50,100.61,0.07,1764.85,1456.34,0.07
1,2018-04-10,AALR3,ALLIAR,175,AALR,2018,2018-03-28 20:20:31,2017-12-31,118.29,507.64,70.34,0.04,1794.50,1286.86,0.05
2,2019-04-10,AALR3,ALLIAR,156,AALR,2019,2019-03-19 19:58:58,2018-12-31,118.29,532.35,137.29,0.07,1731.81,1199.46,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,2022-04-11,VITT3,VITTIA,232,VITT,2022,2022-03-18 12:29:36,2021-12-31,143.05,159.80,142.84,0.22,1753.78,1593.98,0.09
1949,2022-04-11,VVEO3,VIVEO,871,VVEO,2022,2022-03-10 18:22:56,2021-12-31,286.12,-76.53,492.30,0.24,4506.44,4582.97,0.11
1950,2022-04-11,VVEO3,VIVEO,871,VVEO,2022,2022-03-30 18:25:34,2021-12-31,286.12,-76.53,492.30,0.24,4506.44,4582.97,0.11


In [17]:
# Existem algumas empresas, como a COGN3, com baixo valor de mercado e excesso de
# dívida líquida resultando em um EV negativo e, consequentemente, um EY (EBIT/EV) 
# negativo ou distorcido (tendendo a infinito). Apesar do lívro não deixar isso claro,
# iremos remover essas empresas da seleção.
df_mag.query('enterprise_value > 100', inplace=True)
print('Number of companies available for backtesting', df_mag.codneg.nunique())

Number of companies available for backtesting 258


In [18]:
# Remover revisões de DFPs publicadas no mesmo dia ou posteriores ao corte
# No livro, o corte é de uma semana
df_mag.query('doc_env.dt.date < datneg')

,datneg,codneg,nomres,totneg,codemi,year,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield
0,2017-04-10,AALR3,ALLIAR,315,AALR,2017,2017-03-22 23:42:13,2016-12-31,114.90,308.50,100.61,0.07,1764.85,1456.34,0.07
1,2018-04-10,AALR3,ALLIAR,175,AALR,2018,2018-03-28 20:20:31,2017-12-31,118.29,507.64,70.34,0.04,1794.50,1286.86,0.05
2,2019-04-10,AALR3,ALLIAR,156,AALR,2019,2019-03-19 19:58:58,2018-12-31,118.29,532.35,137.29,0.07,1731.81,1199.46,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1948,2022-04-11,VITT3,VITTIA,232,VITT,2022,2022-03-18 12:29:36,2021-12-31,143.05,159.80,142.84,0.22,1753.78,1593.98,0.09
1949,2022-04-11,VVEO3,VIVEO,871,VVEO,2022,2022-03-10 18:22:56,2021-12-31,286.12,-76.53,492.30,0.24,4506.44,4582.97,0.11
1950,2022-04-11,VVEO3,VIVEO,871,VVEO,2022,2022-03-30 18:25:34,2021-12-31,286.12,-76.53,492.30,0.24,4506.44,4582.97,0.11


In [19]:
# Manter somente a DFP mais recente ao corte para cada ativo
df_mag.sort_values('doc_env', inplace=True)
df_mag.drop_duplicates(subset=['codneg', 'year'], keep='last', inplace=True)
df_mag

,datneg,codneg,nomres,totneg,codemi,year,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield
1348,2011-04-11,TOTS3,TOTVS,244,TOTS,2011,2011-01-31 19:05:59,2010-12-31,31.46,179.42,211.67,0.26,994.10,814.68,0.26
784,2011-04-11,LREN3,LOJAS RENNER,3604,LREN,2011,2011-02-16 19:53:52,2010-12-31,122.35,-27.16,404.47,0.41,6682.70,6709.86,0.06
1834,2011-04-11,NETC4,NET,20,NETC,2011,2011-02-18 10:24:59,2010-12-31,342.96,1356.69,658.06,0.13,4897.53,3540.83,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1712,2022-04-11,PLPL3,PLANOEPLANO,1098,PLPL,2022,2022-04-19 17:17:28,2021-12-31,204.26,114.60,163.76,0.38,614.81,500.21,0.33
1644,2022-04-11,ALLD3,ALLIED,273,ALLD,2022,2022-04-27 12:11:23,2021-12-31,93.22,-338.12,407.51,0.35,1382.47,1720.59,0.24
1943,2022-04-11,RECV3,PETRORECSA,2648,RECV,2022,2022-06-02 17:08:52,2021-12-31,248.52,-61.52,284.61,0.16,5743.23,5804.75,0.05


In [20]:
# Remover o ativo menos líquido da empresa no ano pelo critério do núm. de negociações
df_mag.sort_values(by=['year', 'codemi', 'totneg'], inplace=True)
df_mag.drop_duplicates(
    subset=['codemi', 'year'], keep='last', inplace=True, ignore_index=True
)
# A coluna 'codemi' não será mais necessária
df_mag.drop(columns='codemi', inplace=True)
print('Number of companies available for backtesting', df_mag.codneg.nunique())
df_mag

Number of companies available for backtesting 239


,datneg,codneg,nomres,totneg,year,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield
0,2011-04-11,AEDU3,ANHANGUERA,978,2011,2011-05-26 11:54:58,2010-12-31,145.69,-571.50,157.01,0.11,5537.69,6109.18,0.03
1,2011-04-11,ALPA4,ALPARGATAS,158,2011,2011-03-30 16:55:23,2010-12-31,353.46,-358.70,324.23,0.33,4011.73,4370.43,0.07
2,2011-04-11,ALSC3,ALIANSCE,102,2011,2011-03-29 11:28:24,2010-12-31,139.47,-141.72,99.44,0.09,1941.38,2083.10,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,2022-04-11,WEGE3,WEG,25960,2022,2022-02-16 07:05:37,2021-12-31,4197.32,-1428.02,4158.34,0.33,138427.55,139855.57,0.03
1075,2022-04-11,WLMM4,WLM IND COM,14,2022,2022-03-22 22:18:18,2021-12-31,36.41,-152.00,136.13,0.35,1258.13,1410.13,0.10
1076,2022-04-11,YDUQ3,YDUQS PART,10252,2022,2022-03-15 18:09:20,2021-12-31,309.09,3692.84,546.59,0.08,6092.14,2399.31,0.23


In [21]:
# O Livro fala em empresas com pelos menos USD 50 milhões de valor de mercado
# Remover empresas com menos de R$ 250 milhões de valor de mercado
df_mag.query('market_cap > 250', inplace=True)
# Remover ações com baixíssima liquidez no dia
df_mag.query('totneg > 100', inplace=True)
# A coluna 'totneg' não será mais necessária
df_mag.drop(columns='totneg', inplace=True)
print('Number of companies available for backtesting', df_mag.codneg.nunique())
df_mag

Number of companies available for backtesting 214


,datneg,codneg,nomres,year,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield
0,2011-04-11,AEDU3,ANHANGUERA,2011,2011-05-26 11:54:58,2010-12-31,145.69,-571.50,157.01,0.11,5537.69,6109.18,0.03
1,2011-04-11,ALPA4,ALPARGATAS,2011,2011-03-30 16:55:23,2010-12-31,353.46,-358.70,324.23,0.33,4011.73,4370.43,0.07
2,2011-04-11,ALSC3,ALIANSCE,2011,2011-03-29 11:28:24,2010-12-31,139.47,-141.72,99.44,0.09,1941.38,2083.10,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,2022-04-11,VVEO3,VIVEO,2022,2022-03-30 18:25:34,2021-12-31,286.12,-76.53,492.30,0.24,4506.44,4582.97,0.11
1074,2022-04-11,WEGE3,WEG,2022,2022-02-16 07:05:37,2021-12-31,4197.32,-1428.02,4158.34,0.33,138427.55,139855.57,0.03
1076,2022-04-11,YDUQ3,YDUQS PART,2022,2022-03-15 18:09:20,2021-12-31,309.09,3692.84,546.59,0.08,6092.14,2399.31,0.23


In [22]:
df_mag['rank_roic'] = (df_mag
    .groupby(by=['year'])['roic']
    .rank(method='dense', ascending=False)
)
df_mag['rank_ey'] = (df_mag
    .groupby(by=['year'])['earnings_yield']
    .rank(method='dense', ascending=False)
)
df_mag['ranks_sum'] = df_mag['rank_roic'] + df_mag['rank_ey']
df_mag['rank_final'] = (df_mag
    .groupby(by=['year'])['ranks_sum']
    .rank(method='first', ascending=True)
)
cols_integer = ['rank_roic', 'rank_ey', 'ranks_sum', 'rank_final']
df_mag[cols_integer] = df_mag[cols_integer].astype(int)
df_mag

,datneg,codneg,nomres,year,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_roic,rank_ey,ranks_sum,rank_final
0,2011-04-11,AEDU3,ANHANGUERA,2011,2011-05-26 11:54:58,2010-12-31,145.69,-571.50,157.01,0.11,5537.69,6109.18,0.03,44,57,101,56
1,2011-04-11,ALPA4,ALPARGATAS,2011,2011-03-30 16:55:23,2010-12-31,353.46,-358.70,324.23,0.33,4011.73,4370.43,0.07,8,39,47,18
2,2011-04-11,ALSC3,ALIANSCE,2011,2011-03-29 11:28:24,2010-12-31,139.47,-141.72,99.44,0.09,1941.38,2083.10,0.05,51,52,103,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,2022-04-11,VVEO3,VIVEO,2022,2022-03-30 18:25:34,2021-12-31,286.12,-76.53,492.30,0.24,4506.44,4582.97,0.11,40,70,110,52
1074,2022-04-11,WEGE3,WEG,2022,2022-02-16 07:05:37,2021-12-31,4197.32,-1428.02,4158.34,0.33,138427.55,139855.57,0.03,22,120,142,76
1076,2022-04-11,YDUQ3,YDUQS PART,2022,2022-03-15 18:09:20,2021-12-31,309.09,3692.84,546.59,0.08,6092.14,2399.31,0.23,102,46,148,78


In [23]:
df_mag.sort_values(by=['year', 'rank_final'], inplace=True)
# Renomear a coluna 'datneg'
df_mag.rename(columns={'datneg': 'cutoff_date'}, inplace=True)
# A coluna 'year' já está implícita na coluna 'cutoff_date' e não será mais usada
df_mag.drop(columns=['year'], inplace=True)
# Remover colunas de cálculo intemediário
df_mag.drop(columns=['rank_roic', 'rank_ey', 'ranks_sum'], inplace=True)
df_mag

,cutoff_date,codneg,nomres,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_final
13,2011-04-11,BRPR3,BR PROPERT,2011-03-21 17:07:01,2010-12-31,139.51,1597.73,1208.17,0.28,2433.09,835.35,1.45,1
83,2011-04-11,TOTS3,TOTVS,2011-01-31 19:05:59,2010-12-31,31.46,179.42,211.67,0.26,994.10,814.68,0.26,2
24,2011-04-11,ECOR3,ECORODOVIAS,2011-05-16 18:50:28,2010-12-31,558.70,898.67,969.45,0.36,7480.98,6582.31,0.15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,2022-04-11,DASA3,DASA,2022-03-29 12:56:36,2021-12-31,560.51,4032.79,88.52,0.01,12953.41,8920.62,0.01,133
968,2022-04-11,ELMD3,ELETROMIDIA,2022-03-29 19:03:12,2021-12-31,139.14,28.30,9.11,0.01,2177.62,2149.31,0.00,134
1005,2022-04-11,LWSA3,LOCAWEB,2022-03-29 15:08:01,2021-12-31,589.58,-1480.19,8.53,0.01,5052.71,6532.91,0.00,135


In [24]:
df_mag.sort_values('rank_final')

,cutoff_date,codneg,nomres,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_final
13,2011-04-11,BRPR3,BR PROPERT,2011-03-21 17:07:01,2010-12-31,139.51,1597.73,1208.17,0.28,2433.09,835.35,1.45,1
368,2015-04-10,ECOR3,ECORODOVIAS,2015-03-17 19:39:00,2014-12-31,558.70,3525.83,1109.42,0.21,5061.81,1535.98,0.72,1
527,2017-04-10,PRIO3,PETRORIO,2017-03-28 17:48:01,2016-12-31,13.19,-539.18,259.08,0.88,553.09,1092.27,0.24,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,2022-04-11,DASA3,DASA,2022-03-29 12:56:36,2021-12-31,560.51,4032.79,88.52,0.01,12953.41,8920.62,0.01,133
968,2022-04-11,ELMD3,ELETROMIDIA,2022-03-29 19:03:12,2021-12-31,139.14,28.30,9.11,0.01,2177.62,2149.31,0.00,134
1005,2022-04-11,LWSA3,LOCAWEB,2022-03-29 15:08:01,2021-12-31,589.58,-1480.19,8.53,0.01,5052.71,6532.91,0.00,135


In [25]:
# Verificar a situação da Petrobras antes do corte para teste
df_mag.query('codneg == "PETR4"')

,cutoff_date,codneg,nomres,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_final
65,2011-04-11,PETR4,PETROBRAS,2011-03-04 17:04:08,2010-12-31,13044.50,61574.76,45574.59,0.12,360419.45,298844.70,0.15,33
686,2019-04-10,PETR4,PETROBRAS,2019-02-27 19:45:27,2018-12-31,13044.50,268824.00,64876.00,0.12,378942.64,110118.64,0.59,19
1024,2022-04-11,PETR4,PETROBRAS,2022-02-23 22:36:50,2021-12-31,13044.50,265778.00,210831.00,0.32,442730.23,176952.23,1.19,6


In [26]:
# Select only the first 30 companies in the magic rank for each year
df_mag.query('rank_final <= 30', inplace=True)
df_mag.reset_index(drop=True, inplace=True)
print('Number of selected companies for backtesting', df_mag.codneg.nunique())
df_mag

Number of selected companies for backtesting 126


,cutoff_date,codneg,nomres,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_final
0,2011-04-11,BRPR3,BR PROPERT,2011-03-21 17:07:01,2010-12-31,139.51,1597.73,1208.17,0.28,2433.09,835.35,1.45,1
1,2011-04-11,TOTS3,TOTVS,2011-01-31 19:05:59,2010-12-31,31.46,179.42,211.67,0.26,994.10,814.68,0.26,2
2,2011-04-11,ECOR3,ECORODOVIAS,2011-05-16 18:50:28,2010-12-31,558.70,898.67,969.45,0.36,7480.98,6582.31,0.15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,2022-04-11,PDTC3,PADTEC,2022-03-15 18:02:05,2021-12-31,78.45,20.34,55.34,0.37,401.66,381.33,0.15,28
358,2022-04-11,POSI3,POSITIVO TEC,2022-03-30 18:17:00,2021-12-31,141.80,541.60,303.15,0.18,1210.97,669.38,0.45,29
359,2022-04-11,DXCO3,DEXCO,2022-02-09 20:32:27,2021-12-31,760.96,2448.35,1891.39,0.23,10021.88,7573.54,0.25,30


In [27]:
df_mag.to_csv('../data/2_magic_stocks.csv', index=False)

In [28]:
df_mag = pd.read_csv('../data/2_magic_stocks.csv')
df_mag

,cutoff_date,codneg,nomres,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_final
0,2011-04-11,BRPR3,BR PROPERT,2011-03-21 17:07:01,2010-12-31,139.51,1597.73,1208.17,0.28,2433.09,835.35,1.45,1
1,2011-04-11,TOTS3,TOTVS,2011-01-31 19:05:59,2010-12-31,31.46,179.42,211.67,0.26,994.10,814.68,0.26,2
2,2011-04-11,ECOR3,ECORODOVIAS,2011-05-16 18:50:28,2010-12-31,558.70,898.67,969.45,0.36,7480.98,6582.31,0.15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,2022-04-11,PDTC3,PADTEC,2022-03-15 18:02:05,2021-12-31,78.45,20.34,55.34,0.37,401.66,381.33,0.15,28
358,2022-04-11,POSI3,POSITIVO TEC,2022-03-30 18:17:00,2021-12-31,141.80,541.60,303.15,0.18,1210.97,669.38,0.45,29
359,2022-04-11,DXCO3,DEXCO,2022-02-09 20:32:27,2021-12-31,760.96,2448.35,1891.39,0.23,10021.88,7573.54,0.25,30


In [29]:
# Empresas que mais aparecem no ranking
df_mag.codneg.value_counts().head(5)

KLBN4    10
VALE3     9
BRML3     8
NATU3     8
TGMA3     8
Name: codneg, dtype: int64

In [30]:
# Check 2022 data
df_22 = df_mag.query('cutoff_date > "2022-01-01"')[['codneg', 'nomres', 'roic', 'earnings_yield']].reset_index(drop=True)
df_22.index += 1
df_22[['roic', 'earnings_yield']] = df_22[['roic', 'earnings_yield']].round(2)
print(df_22.to_markdown(mode='github'))

|    | codneg   | nomres       |   roic |   earnings_yield |
|---:|:---------|:-------------|-------:|-----------------:|
|  1 | BRKM3    | BRASKEM      |   0.9  |             2.04 |
|  2 | GOAU4    | GERDAU MET   |   0.42 |             3.37 |
|  3 | BRAP4    | BRADESPAR    |   1.11 |             0.61 |
|  4 | USIM3    | USIMINAS     |   0.49 |             0.72 |
|  5 | TASA4    | TAURUS ARMAS |   0.82 |             0.4  |
|  6 | PETR4    | PETROBRAS    |   0.32 |             1.19 |
|  7 | GGBR4    | GERDAU       |   0.42 |             0.49 |
|  8 | JBSS3    | JBS          |   0.28 |             1.4  |
|  9 | VALE3    | VALE         |   0.68 |             0.3  |
| 10 | BEEF3    | MINERVA      |   0.3  |             0.95 |
| 11 | CMIN3    | CSNMINERACAO |   1.22 |             0.25 |
| 12 | DEXP3    | DEXXOS PAR   |   0.33 |             0.55 |
| 13 | ENAT3    | ENAUTA PART  |   1.1  |             0.26 |
| 14 | SUZB3    | SUZANO S.A.  |   0.25 |             1.17 |
| 15 | EUCA4    | EUCATE

In [30]:
df_mag.query('codneg == "PRIO3"')

,cutoff_date,codneg,nomres,doc_env,per_fim,shares_outstanding,net_debt,ebit,roic,market_cap,enterprise_value,earnings_yield,rank_final
180,2017-04-10,PRIO3,PETRORIO,2017-03-28 17:48:01,2016-12-31,13.19,-539.18,259.08,0.88,553.09,1092.27,0.24,1.00
272,2020-04-09,PRIO3,PETRORIO,2020-08-28 02:11:58,2019-12-31,143.19,1572.36,878.26,0.23,3800.15,2227.79,0.39,3.00
